In [1]:
# 1
from utils import (download_csv, calculate_splits, prompt_2_call, 
                   extract_context, prompt_1_call, prompt_3_call, 
                   download_all, dict_cleaning, try_days_param)
import pandas as pd
import time, re
import os
from tqdm import tqdm
from IPython.display import clear_output
import shutil

/Users/bibbi/Desktop/tesi isa/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/bibbi/Desktop/tesi isa/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
df_cyberattacks = pd.read_csv('/Users/bibbi/Desktop/tesi isa/tesi/Data Breaches.csv')
df_cyberattacks = df_cyberattacks.iloc[1:].sort_values('year   ', ascending = True)
df_cyberattacks['records lost'] = (
    df_cyberattacks['records lost']
    .str.replace(',', '', regex=False)   
    .astype(float)                       
    .astype('Int64')               
)

df_cyberattacks = df_cyberattacks.sort_values('records lost', ascending = False)
df_cyberattacks.head()

,organisation,alternative name,records lost,year,date,story,sector,method,interesting story,data sensitivity,displayed records,Unnamed: 11,source name,1st source link,2nd source link,ID
36,Linkedin,NaN,700000000,2021,Jul 2021,The hacker appears to have misused the officia...,web,hacked,NaN,1,"700,000,000",NaN,9 to 5 mac,https://9to5mac.com/2021/06/29/linkedin-breach/,NaN,385.0
143,Aadhaar,"India's national, biometric government ID data...",550000000,2018,Mar 2018,India's biometric database was breached via a ...,government,poor security,NaN,4,"1,100,000,000",NaN,ZDNet,http://www.zdnet.com/article/another-data-leak...,NaN,276.0
269,Yahoo,NaN,550000000,2013,Dec 2016,A 2013 attack was eventually disclosed in 2016...,web,hacked,NaN,2,"1,000,000,000",NaN,"NY Times, BBC",http://www.nytimes.com/2016/12/14/technology/y...,https://www.bbc.co.uk/news/business-41493494,150.0
50,Facebook,NaN,533000000,2021,Mar 2021,"Phone numbers, full names, locations, email ad...",tech,hacked,y,1,"533,000,000",NaN,Business Insider,https://www.businessinsider.com/stolen-data-of...,NaN,371.0
168,Spambot,NaN,520000000,2017,Aug 2017,A misconfigured spambot leaked email addresses...,web,poor security,NaN,4,"711,000,000",NaN,The Guardian,https://www.theguardian.com/technology/2017/au...,NaN,251.0


In [ ]:
company = 'Microsoft'
date = 'May 2021'
days_param = 5
download_csv(company, date, days_param)

Fetching for company Microsoft from 2021-04-15 to 2021-06-30
[2021-04-15] querying 20210415000000 → 20210420000000


0it [00:00, ?it/s]

Error Status 429!
Elapsed time before 429 error : 1.9961321353912354
Number of calls before 429 error : 1
Retrying for Error 429!


0it [00:08, ?it/s]


KeyboardInterrupt: 

In [ ]:
data_folder = '/Users/bibbi/Desktop/tesi isa/tesi/data'
for folder in os.listdir(data_folder):
    print(f'------- Company: {os.path.basename(folder)} -------')
    folder_path = os.path.join(data_folder, folder)
    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        df_single = pd.read_csv(file_path)
        print(f'Number of elements for file {file}: {len(df_single)}')

------- Company: Uber -------
Number of elements for file gdelt__Nov_2017.csv: 4753
------- Company: Twitter -------
Number of elements for file gdelt__May_2018.csv: 3976
------- Company: Microsoft -------
Number of elements for file gdelt__Jan_2020.csv: 7039
------- Company: Facebook -------
Number of elements for file gdelt__Mar_2021.csv: 12820
Number of elements for file gdelt__Sep_2019.csv: 11630
------- Company: Linkedin -------
Number of elements for file gdelt_Jul_2021.csv: 466


In [15]:
company = 'Uber'
date = 'Nov_2017'
file_name = f'gdelt__{date}.csv'
df = pd.read_csv(f'{data_folder}/{company}/{file_name}')
print(f'Lenght of the file : {len(df)}')
df.head()

Lenght of the file : 4753


,URL,MobileURL,Date,Title,The specified phrase is too short.,Parentheses may only be used around OR'd statements.
0,https://www.recode.net/2017/10/16/16486418/for...,https://www.recode.net/platform/amp/2017/10/16...,2017-10-17 01:00:00,A former Uber exec has filed to dismiss a case...,NaN,NaN
1,https://www.elheraldodechihuahua.com.mx/republ...,https://www.elheraldodechihuahua.com.mx/republ...,2017-10-16 00:45:00,Ingresan al penal a chofer de Uber por feminic...,NaN,NaN
2,https://www.elsoldepuebla.com.mx/republica/ing...,https://www.elsoldepuebla.com.mx/republica/ing...,2017-10-15 23:15:00,Ingresan al penal a chofer de Uber por feminic...,NaN,NaN
3,https://www.elmanana.com/suspendenserviciodeub...,https://www.elmanana.com/amp/suspendenservicio...,2017-10-18 05:00:00,Suspenden servicio de Uber en Puebla,NaN,NaN
4,http://www.zocalo.com.mx/new_site/articulo/anu...,NaN,2017-10-17 19:45:00,Anuncian suspensión de Uber en Puebla [ Nacion...,NaN,NaN


In [ ]:
skip = 2500
count = 1
for start in list(range(0,len(df), skip)):
    print(f'Starting batch {count} - from line {start} to line {start+skip}')
    download_all(company, file_name, n = count, start = start, skip = skip, devided = True)
    count += 1

In [ ]:
count = 1
skip = 2500
df_new = pd.DataFrame()
data_extracted_folder = f'/Users/bibbi/Desktop/tesi isa/tesi/data_extracted/{company}'

for _ in range(0,len(df), skip):
    file_name_new = f'{count}_extracted_{file_name}'
    print(file_name_new)
    dir_file = os.path.join(data_extracted_folder, file_name_new)
    df_sample = pd.read_csv(dir_file)
    print(f'length of file number {count} : {len(df_sample)}')
    df_new = pd.concat([df_new, df_sample])
    count += 1

output_dir = os.path.join(data_extracted_folder, f'extracted_{file_name}')
df_new.to_csv(output_dir)
df = df_new
df.head()

1_extracted_gdelt__Mar_2021.csv
length of file number 1 : 2500
2_extracted_gdelt__Mar_2021.csv
length of file number 2 : 2500
3_extracted_gdelt__Mar_2021.csv
length of file number 3 : 2500
4_extracted_gdelt__Mar_2021.csv
length of file number 4 : 2500
5_extracted_gdelt__Mar_2021.csv
length of file number 5 : 2500
6_extracted_gdelt__Mar_2021.csv
length of file number 6 : 320


,URL,MobileURL,Date,Title,The specified phrase is too short.,Parentheses may only be used around OR'd statements.,text_full,Facebook,cyberattack,keylogger,...,sniffing,overwriting,encryption,interface,encrypted,sanitization,nontechnical,exploiter,disinformation,ransomware
0,https://www.al.com/news/2021/02/facebook-plans...,https://www.al.com/news/2021/02/facebook-plans...,2021-02-16 17:00:00,Facebook plans expansion to Huntsville data ce...,NaN,NaN,Facebook plans expansion to Huntsville data ce...,18,0,0,...,0,0,0,0,0,0,0,0,0,0
1,https://hannity.com/media-room/ministry-of-tru...,https://hannity.com/media-room/ministry-of-tru...,2021-02-18 19:30:00,MINISTRY of TRUTH ? Facebook to Flag Posts on ...,NaN,NaN,MINISTRY of TRUTH? Facebook to Flag Posts on C...,16,0,0,...,0,0,0,0,0,0,0,0,0,0
2,http://www.msn.com/en-us/news/politics/hillico...,NaN,2021-02-19 00:00:00,Hillicon Valley : Congress prepares to hold he...,NaN,NaN,MSN,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,https://thehill.com/policy/technology/overnigh...,https://thehill.com/policy/technology/overnigh...,2021-02-19 00:45:00,Hillicon Valley : Congress prepares to hold he...,NaN,NaN,[Playwright Error] BrowserType.launch: Executa...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,https://www.euractiv.com/section/digital/news/...,NaN,2021-02-19 15:15:00,"Digital Brief , powered by Facebook : united k...",NaN,NaN,"Digital Brief, powered by Facebook: UK data ad...",6,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
df = pd.read_csv(f'/Users/bibbi/Desktop/tesi isa/tesi/data_extracted/{company}/extracted_gdelt__{date}.csv')
df.head()

,URL,MobileURL,Date,Title,The specified phrase is too short.,Parentheses may only be used around OR'd statements.,text_full,Uber,cyberattack,keylogger,...,sniffing,overwriting,encryption,interface,encrypted,sanitization,nontechnical,exploiter,disinformation,ransomware
0,https://www.recode.net/2017/10/16/16486418/for...,https://www.recode.net/platform/amp/2017/10/16...,2017-10-17 01:00:00,A former Uber exec has filed to dismiss a case...,NaN,NaN,[Playwright Error] BrowserType.launch: Executa...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,https://www.elheraldodechihuahua.com.mx/republ...,https://www.elheraldodechihuahua.com.mx/republ...,2017-10-16 00:45:00,Ingresan al penal a chofer de Uber por feminic...,NaN,NaN,[Playwright Error] BrowserType.launch: Executa...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,https://www.elsoldepuebla.com.mx/republica/ing...,https://www.elsoldepuebla.com.mx/republica/ing...,2017-10-15 23:15:00,Ingresan al penal a chofer de Uber por feminic...,NaN,NaN,[Playwright Error] BrowserType.launch: Executa...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,https://www.elmanana.com/suspendenserviciodeub...,https://www.elmanana.com/amp/suspendenservicio...,2017-10-18 05:00:00,Suspenden servicio de Uber en Puebla,NaN,NaN,Suspenden servicio de Uber en Puebla En Línea ...,6,0,0,...,0,0,0,0,0,0,0,0,0,0
4,http://www.zocalo.com.mx/new_site/articulo/anu...,NaN,2017-10-17 19:45:00,Anuncian suspensión de Uber en Puebla [ Nacion...,NaN,NaN,Anuncian suspensión de Uber en Puebla – Periód...,6,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
keywords = [
    "cyberattack", "keylogger", "cyberattacks", "cyber", "subdomains", "driveby", "modifiable", "malicious",
    "malware", "countermeasures", "cybercrime", "phishing", "vishing", "weaponised", "multifactor", "backdoors",
    "opensource", "runtimes", "unencrypted", "hacktivists", "embed", "reconfigurations", "unpatched", "gapped",
    "firewalls", "timezones", "rootkits", "untrusted", "fuzzing", "configured", "wideband", "remotely", "flaw",
    "alerts", "eavesdropping", "cybersecurity", "validations", "simulating", "router", "hackers",
    "microcontroller authenticator", "intrusion", "cryptographically", "unsecure", "licenced", "directive",
    "spoofing", "sniffing", "overwriting", "encryption", "interface", "encrypted", "sanitization",
    "nontechnical", "exploiter", "disinformation", "ransomware"
]

df['value_keywords_count'] = df[keywords].sum(axis=1)
df['flag_cyberattack_manual'] = ((df['value_keywords_count']>=10) & (df[company] >= 5)).astype(int)
df.head()

,URL,MobileURL,Date,Title,The specified phrase is too short.,Parentheses may only be used around OR'd statements.,text_full,Twitter,cyberattack,keylogger,...,encryption,interface,encrypted,sanitization,nontechnical,exploiter,disinformation,ransomware,value_keywords_count,flag_cyberattack_manual
0,http://www.profile.ru/hi-tech/internet/item/12...,NaN,2018-04-17 17:15:00,оссийские пользователи столкнулись с перебоям...,NaN,NaN,Российские пользователи столкнулись с перебоям...,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,https://thebulletin.org/what-us-nuclear-policy...,NaN,2018-04-18 22:45:00,"What is US nuclear policy , exactly ?",NaN,NaN,[Playwright Error] BrowserType.launch: Executa...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,http://www.btvi.in/article/read/technology/676...,NaN,2018-04-18 10:45:00,New - algorithm - may - help - locate - fake -...,NaN,NaN,[Playwright Error] BrowserType.launch: Executa...,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,https://www.arlnow.com/2018/04/19/,NaN,2018-04-19 16:30:00,"April 19 , 2018",NaN,NaN,"April 19, 2018 | ARLnow.com Get Newsletter Hom...",5,0,0,...,0,0,0,0,0,0,0,0,3,0
4,https://www.lanacion.com.ar/2126473-nosotros-c...,https://www.lanacion.com.ar/2126473-nosotros-c...,2018-04-18 07:00:00,¿ Nosotros creamos este monstruo ? Cómo Twitt...,NaN,NaN,[Playwright Error] BrowserType.launch: Executa...,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
splits_prompt_2 = calculate_splits(df, prompt='prompt_2.txt', token_limit=50000)
print(splits_prompt_2)

flag_list = []
for split_start, split_end in tqdm(splits_prompt_2):
    list_text = df['text_full'].iloc[split_start: split_end].to_list()

    try:
        cyberattack_flags = prompt_2_call(list_text, company)
        if len(cyberattack_flags)!= len(list_text):
            raise ValueError
    except:
        n_times = split_end - split_start
        cyberattack_flags = [0] * n_times
    time.sleep(4)
    flag_list += cyberattack_flags
    clear_output(wait=True)

print(f'Total elements outputted : {len(flag_list)}')

df['flag_cyberattack_gemini'] = flag_list
folder_prompt2_path = f'/Users/bibbi/Desktop/tesi isa/tesi/prompts_output/{company}/{date}'
os.makedirs(folder_prompt2_path, exist_ok=True)
df.to_csv(f'/Users/bibbi/Desktop/tesi isa/tesi/prompts_output/{company}/{date}/df_prompt_2.csv')
df.head()

100%|██████████| 265/265 [47:07<00:00, 10.67s/it]


Total elements outputted : 3976


OSError: Cannot save file into a non-existent directory: '/Users/bibbi/Desktop/tesi isa/tesi/prompts_output/Twitter/May_2018'

In [ ]:
company = 'Twitter'
date = 'May_2018'
df = pd.read_csv(f'/Users/bibbi/Desktop/tesi isa/tesi/prompts_output/{company}/{date}/df_prompt_2.csv')
df.head()

,Unnamed: 0,URL,MobileURL,Date,Title,The specified phrase is too short.,Parentheses may only be used around OR'd statements.,text_full,Facebook,cyberattack,...,sanitization,nontechnical,exploiter,disinformation,ransomware,value_keywords_count,flag_cyberattack_manual,flag_cyberattack_gemini,has_value_manual,value_context_manual
0,0,https://www.al.com/news/2021/02/facebook-plans...,https://www.al.com/news/2021/02/facebook-plans...,2021-02-16 17:00:00,Facebook plans expansion to Huntsville data ce...,NaN,NaN,Facebook plans expansion to Huntsville data ce...,18,0,...,0,0,0,0,0,1,0,0,True,project; Price could now top $1 billion - al....
1,1,https://hannity.com/media-room/ministry-of-tru...,https://hannity.com/media-room/ministry-of-tru...,2021-02-18 19:30:00,MINISTRY of TRUTH ? Facebook to Flag Posts on ...,NaN,NaN,MINISTRY of TRUTH? Facebook to Flag Posts on C...,16,0,...,0,0,0,0,0,2,0,0,False,NaN
2,2,http://www.msn.com/en-us/news/politics/hillico...,NaN,2021-02-19 00:00:00,Hillicon Valley : Congress prepares to hold he...,NaN,NaN,MSN,0,0,...,0,0,0,0,0,0,0,0,False,NaN
3,3,https://thehill.com/policy/technology/overnigh...,https://thehill.com/policy/technology/overnigh...,2021-02-19 00:45:00,Hillicon Valley : Congress prepares to hold he...,NaN,NaN,[Playwright Error] BrowserType.launch: Executa...,0,0,...,0,0,0,0,0,0,0,0,False,NaN
4,4,https://www.euractiv.com/section/digital/news/...,NaN,2021-02-19 15:15:00,"Digital Brief , powered by Facebook : united k...",NaN,NaN,"Digital Brief, powered by Facebook: UK data ad...",6,0,...,0,0,0,0,0,0,0,0,True,p companies to pay government 15% cut of China...


In [12]:
print('Number of same guessing for gemini manual check : ',len(df[df['flag_cyberattack_gemini'] == df['flag_cyberattack_manual']]))
print(f'Number of 1s (articles related to cyber incidents) for gemini : {len(df[df["flag_cyberattack_gemini"] == 1])}')
print(f'Number of 1s (articles related to cyber incidents) for manual seach : {len(df[df["flag_cyberattack_manual"] == 1])}')
print(f'Number of 1s (articles related to cyber incidents) with the additional constraints of manual = gemini : {len(df[(df["flag_cyberattack_gemini"] == df["flag_cyberattack_manual"]) & (df["flag_cyberattack_gemini"] == 1)])}')

Number of same guessing for gemini manual check :  9923
Number of 1s (articles related to cyber incidents) for gemini : 3552
Number of 1s (articles related to cyber incidents) for manual seach : 861
Number of 1s (articles related to cyber incidents) with the additional constraints of manual = gemini : 758


In [ ]:
df = df[df["flag_cyberattack_gemini"] == 1]

value_pattern = r'(\$[\d,.]+|€[\d,.]+|£[\d,.]+|\b\d+(?:\.\d+)?\s?(?:million|billion|thousand|k|%)\b|\bUSD\s?\d+|\d+\s?(?:percent|%))'

df['has_value_manual'] = df['text_full'].str.contains(value_pattern, flags=re.IGNORECASE, na=False)
df['value_context_manual'] = df['text_full'].apply(extract_context)
df.head()

/var/folders/lc/yktkwghx4pq7572f0gkp5mbw0000gn/T/ipykernel_88584/2891784926.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df['has_value_manual'] = df['text_full'].str.contains(value_pattern, flags=re.IGNORECASE, na=False)


,Unnamed: 0,URL,MobileURL,Date,Title,The specified phrase is too short.,Parentheses may only be used around OR'd statements.,text_full,Facebook,cyberattack,...,sanitization,nontechnical,exploiter,disinformation,ransomware,value_keywords_count,flag_cyberattack_manual,flag_cyberattack_gemini,has_value_manual,value_context_manual
9,9,https://www.jagran.com/west-bengal/kolkata-sou...,https://m.jagran.com/lite/west-bengal/kolkata-...,2021-02-17 13:15:00,BCCI president Sourav Ganguly wife Dona lodges...,NaN,NaN,West Bengal: BCCI अध्यक्ष सौरव गांगुली की पत्न...,1,0,...,0,0,0,0,0,0,0,1,False,None
15,15,https://www.techtimes.com/articles/257042/2021...,https://www.techtimes.com/amp/articles/257042/...,2021-02-15 13:45:00,WhatsApp : Here How to Protect Your Phone Numb...,NaN,NaN,WhatsApp: Here's How to Protect Your Phone Num...,7,0,...,0,0,0,0,0,5,0,1,True,. WhatsApp is encouraging its 1.5 billion user...
23,23,https://www.akhbarelyoum.dz/ar/200235/298896-2...,NaN,2021-02-16 21:30:00,بلحيمر : ما نُحذّر منه ليس نسجاً من الخيال,NaN,NaN,بلحيمر: ما نُحذّر منه ليس نسجاً من الخيال أخبا...,0,0,...,0,0,0,0,0,0,0,1,False,None
28,28,https://www.heraldsun.com.au/news?nk=c17e48546...,NaN,2021-02-18 03:45:00,HeraldSunSections | Herald Sun,NaN,NaN,News | HeraldSunSections | Herald Sun Read Tod...,2,0,...,0,0,0,0,0,0,0,1,True,ay” for homeowners who’ll get $109 of relief a...
29,29,https://www.heraldsun.com.au/news?nk=582c68db9...,NaN,2021-02-18 04:00:00,HeraldSunSections | Herald Sun,NaN,NaN,News | HeraldSunSections | Herald Sun Read Tod...,0,0,...,0,0,0,0,0,0,0,1,False,None


In [14]:
start_values = 0
splits_prompt_1 = calculate_splits(df.iloc[start_values:], prompt='prompt_1.txt', token_limit=50000)
print(splits_prompt_1)

value_found_list = []
description_of_value_list = []
errori = []
start = 0
for split_start, split_end in tqdm(splits_prompt_1):
    list_text = df['text_full'].iloc[split_start+start_values : split_end+start_values].to_list()
    print((split_start+start_values,split_end+start_values))
    try:
        response = prompt_1_call(list_text)
        if len(response)!= (split_end - split_start):
            raise ValueError
    except Exception as e:
        n_times = split_end - split_start
        print(f'Error: {e} ---- generating automatic list of lenght {n_times}')
        response = [{'value_found': [], 'description_of_value': []}] * n_times
        errori.append((split_start+start_values, split_end+start_values))

    if len(response) != (split_end-split_start):
        raise ValueError
    
    time.sleep(4)
    value_found_list += [entry['value_found'] for entry in response]
    description_of_value_list += [entry['description_of_value'] for entry in response]
    
    if (start+100)>= split_start and (start+100)<split_end:
        df_temporary = df.iloc[start:split_end]
        df_temporary['has_value_gemini'] = value_found_list
        df_temporary['value_description_gemini'] = description_of_value_list
        folder_splits_path = f'/Users/bibbi/Desktop/tesi isa/tesi/prompts_output/{company}/{date}/splits_prompt_1'
        os.makedirs(folder_splits_path, exist_ok=True)
        file_splits_path = os.path.join(folder_splits_path, f'{start_values+start}_{start_values+split_end}_df_prompt_1.csv')
        df_temporary.to_csv(file_splits_path)

        value_found_list = []
        description_of_value_list = []
        start = split_end
        print(f'Elements from {start} to {split_end} saved!')

    clear_output(wait=True)

print(f'Number of Errors occured : {len(errori)}')
print(f'Errors: {errori}')

  1%|          | 1/163 [00:05<14:32,  5.38s/it]

(17, 34)
Error: 'list' object has no attribute 'get' ---- generating automatic list of lenght 17


  1%|          | 1/163 [00:09<24:48,  9.19s/it]


KeyboardInterrupt: 

In [ ]:
folder_path = f'/Users/bibbi/Desktop/tesi isa/tesi/prompts_output/{company}/{date}/splits_prompt_1'
csv_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path)]
dfs = [pd.read_csv(f) for f in csv_files]
df = pd.concat(dfs, ignore_index=True)
df.to_csv(f'/Users/bibbi/Desktop/tesi isa/tesi/prompts_output/{company}/{date}/df_prompt_1.csv')

In [9]:
df.head()

,Unnamed: 0,URL,MobileURL,Date,Title,The specified phrase is too short.,Parentheses may only be used around OR'd statements.,text_full,Facebook,cyberattack,...,disinformation,ransomware,value_keywords_count,flag_cyberattack_manual,flag_cyberattack_gemini,has_value_manual,value_context_manual,has_value_gemini,value_description_gemini,Unnamed: 0.1
0,2276,https://laser1017.iheart.com/content/2021-04-0...,NaN,2021-04-05 20:45:00,Here How To Check If Your Phone Number Was Inv...,NaN,NaN,Here's How To Check If Your Phone Number Was I...,10,0,...,0,0,2,0,1,True,nter for Your Chance to Win a $250 NYX Profess...,[],[],NaN
1,2277,https://1045snx.iheart.com/content/2021-04-05-...,NaN,2021-04-05 20:45:00,Here How To Check If Your Phone Number Was Inv...,NaN,NaN,Here's How To Check If Your Phone Number Was I...,10,0,...,0,0,2,0,1,True,"orite Teacher A Chance To Win $5,000 In Classr...",[],[],NaN
2,2278,https://kcjb910.iheart.com/content/2021-04-05-...,NaN,2021-04-05 20:45:00,Here How To Check If Your Phone Number Was Inv...,NaN,NaN,Here's How To Check If Your Phone Number Was I...,10,0,...,0,0,2,0,1,True,nter for Your Chance to Win a $250 NYX Profess...,[],[],NaN
3,2279,https://powertalk967.iheart.com/content/2021-0...,NaN,2021-04-05 20:45:00,Here How To Check If Your Phone Number Was Inv...,NaN,NaN,Here's How To Check If Your Phone Number Was I...,11,0,...,0,0,2,0,1,True,"orite Teacher A Chance To Win $5,000 In Classr...",[],[],NaN
4,2280,https://700wlw.iheart.com/content/2021-04-05-h...,NaN,2021-04-05 20:45:00,Here How To Check If Your Phone Number Was Inv...,NaN,NaN,Here's How To Check If Your Phone Number Was I...,10,0,...,0,0,2,0,1,True,"orite Teacher A Chance To Win $5,000 In Classr...",[],[],NaN


In [ ]:
company = 'Facebook'
date = 'Mar_2021'
df = pd.read_csv(f'/Users/bibbi/Desktop/tesi isa/tesi/prompts_output/{company}/{date}/df_prompt_1.csv')
df.head()

,Unnamed: 0.2,Unnamed: 0,URL,MobileURL,Date,Title,The specified phrase is too short.,Parentheses may only be used around OR'd statements.,text_full,Facebook,...,disinformation,ransomware,value_keywords_count,flag_cyberattack_manual,flag_cyberattack_gemini,has_value_manual,value_context_manual,has_value_gemini,value_description_gemini,Unnamed: 0.1
0,0,2276,https://laser1017.iheart.com/content/2021-04-0...,NaN,2021-04-05 20:45:00,Here How To Check If Your Phone Number Was Inv...,NaN,NaN,Here's How To Check If Your Phone Number Was I...,10,...,0,0,2,0,1,True,nter for Your Chance to Win a $250 NYX Profess...,[],[],NaN
1,1,2277,https://1045snx.iheart.com/content/2021-04-05-...,NaN,2021-04-05 20:45:00,Here How To Check If Your Phone Number Was Inv...,NaN,NaN,Here's How To Check If Your Phone Number Was I...,10,...,0,0,2,0,1,True,"orite Teacher A Chance To Win $5,000 In Classr...",[],[],NaN
2,2,2278,https://kcjb910.iheart.com/content/2021-04-05-...,NaN,2021-04-05 20:45:00,Here How To Check If Your Phone Number Was Inv...,NaN,NaN,Here's How To Check If Your Phone Number Was I...,10,...,0,0,2,0,1,True,nter for Your Chance to Win a $250 NYX Profess...,[],[],NaN
3,3,2279,https://powertalk967.iheart.com/content/2021-0...,NaN,2021-04-05 20:45:00,Here How To Check If Your Phone Number Was Inv...,NaN,NaN,Here's How To Check If Your Phone Number Was I...,11,...,0,0,2,0,1,True,"orite Teacher A Chance To Win $5,000 In Classr...",[],[],NaN
4,4,2280,https://700wlw.iheart.com/content/2021-04-05-h...,NaN,2021-04-05 20:45:00,Here How To Check If Your Phone Number Was Inv...,NaN,NaN,Here's How To Check If Your Phone Number Was I...,10,...,0,0,2,0,1,True,"orite Teacher A Chance To Win $5,000 In Classr...",[],[],NaN


In [ ]:
import time
import concurrent.futures

default_dict = {
    "Information_Privacy": "",
    "Proprietary_data_loss": "",
    "Business_interruption": "",
    "Cyber_extortion": "",
    "Network_security": "",
    "Financial_fraud": "",
    "Media_content": "",
    "Hardware_bricking": "",
    "Post_breach_security_requirements": "",
    "Reputational_damage": "",
    "Data_affected_or_stolen": "",
    "Alleged_attacker_or_threat_actor": "",
    "Response_by_the_entity": ""
}

start_value = 3213
splits_prompt_3 = calculate_splits(df.iloc[start_value:], prompt='prompt_3.txt', token_limit=40000)
print(splits_prompt_3)

numbered_sequence_of_economic_values = []

for line in range(len(df.iloc[start_value:])):
    value_found = df['has_value_gemini'].iloc[line+start_value]
    value_description_gemini = df['value_description_gemini'].iloc[line+start_value]
    numbered_sequence_of_economic_values.append({'value_found': value_found, 'description_of_value': value_description_gemini})

dict_list = []
start = 0
errori = []
for split_start, split_end in tqdm(splits_prompt_3):
    list_text = df['text_full'].iloc[split_start+start_value : split_end+start_value].to_list()
    list_values = df['has_value_gemini'].iloc[split_start+start_value : split_end+start_value].to_list()
    list_descriptions = df['value_description_gemini'].iloc[split_start+start_value : split_end+start_value].to_list()
    print((split_start+start_value,split_end+start_value))

    try:
        with concurrent.futures.ThreadPoolExecutor() as executor:
            future = executor.submit(
                prompt_3_call, list_text, list_values, list_descriptions, numbered_sequence_of_economic_values
            )
            response = future.result(timeout=300)
    except concurrent.futures.TimeoutError:
        print("Timeout after 5 minutes. Returning default dictionary.")
        response = default_dict.copy()
        errori.append((split_start+start_value, split_end+start_value))
    except Exception as e:
        print(f"Error: {e}")
        response = default_dict.copy()
        errori.append((split_start+start_value, split_end+start_value))
    
    dict_list.append(response)

    if (start+100)>= split_start and (start+100)<split_end:
        df_temporary = pd.DataFrame(dict_list)
        folder_splits_path = f'/Users/bibbi/Desktop/tesi isa/tesi/prompts_output/{company}/{date}/splits_prompt_3'
        os.makedirs(folder_splits_path, exist_ok=True)
        file_splits_path = os.path.join(folder_splits_path, f'{start_value+start}_{start_value+split_end}_df_prompt_3.csv')
        df_temporary.to_csv(file_splits_path)
        
        dict_list = []
        start = split_end
        print(f'Elements from {start} to {split_end} saved!')

    clear_output(wait=True)

print(f'Number of Errors occured : {len(errori)}')
print(f'Errors: {errori}')

100%|██████████| 24/24 [18:30<00:00, 46.26s/it]

Number of Errors occured : 0
Errors: []


In [ ]:
company = 'Facebook'
date = 'Mar_2021'
folder_path = f'/Users/bibbi/Desktop/tesi isa/tesi/prompts_output/{company}/{date}/splits_prompt_3'
csv_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path)]
dfs = [pd.read_csv(f) for f in csv_files]
df.to_csv(f'/Users/bibbi/Desktop/tesi isa/tesi/prompts_output/{company}/{date}/df_prompt_3.csv')
df.head()

,Information_Privacy,Proprietary_data_loss,Business_interruption,Cyber_extorsion,Network_security,Financial_fraud,Media_content,Hardware_bricking,Post_breach_security_requirements,Reputational_damage,Data_affected_or_stolen,Alleged_attacker_or_threat_actor,Response_by_the_entity,Cyber_extortion,Note
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Personal data of over 533 million Facebook use...,"An unnamed hacker (one claimed to be Russian, ...",Facebook stated that the data was 'old' (from ...,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Personal data of over 533 million Facebook use...,Unspecified hackers or bad actors exploited a ...,Facebook stated that the leaked data was 'old ...,NaN,NaN
3,"$6,090,909",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Personal information of 533 million Facebook u...,An unspecified hacker or group of hackers who ...,Facebook stated that the data was 'old' (from ...,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Malware was used to infect devices (computers,...","Chinese hackers, identified as 'Earth Empusa' ...",Facebook took down approximately 100 accounts ...,NaN,NaN


In [ ]:
list_of_dicts = df.to_dict(orient="records")
final_dict = {k: [d[k] for d in list_of_dicts if d.get(k)] for k in list_of_dicts[0]}
final_dict = dict_cleaning(final_dict)
df_single_attack = pd.DataFrame(final_dict)
df_single_attack.head()

,Information_Privacy,Proprietary_data_loss,Business_interruption,Cyber_extorsion,Network_security,Financial_fraud,Media_content,Hardware_bricking,Post_breach_security_requirements,Reputational_damage,Data_affected_or_stolen,Alleged_attacker_or_threat_actor,Response_by_the_entity,Cyber_extortion,Note
0,"[$6,090,909, $5,000,000,000, $5,000,000,000, $...",[],"[Tens of millions USD - $30,000,000 - $3,000,000]","[$20, tens of thousands of dollars, $35,100, $...","[$5,000, Costs associated with finding and fix...","[$2,087.5 (estimated user losses from scams le...",[],[],"[$57,000,000, $57,000,000, $57,000,000, 570000...","[$60,000,000,000 (market cap wiped off due to ...",[Personal data of over 533 million Facebook us...,"[An unnamed hacker (one claimed to be Russian,...",[Facebook stated that the data was 'old' (from...,[],[The provided text passages and economic value...


In [48]:
final_file_path = f'/Users/bibbi/Desktop/tesi isa/tesi/prompts_output/{company}/{date}/all_attacks.csv'

if os.path.exists(final_file_path):
    df_original = pd.read_csv(final_file_path)
    df_original['Company'] = company
    df_original['Attack Date'] = date
    df_original = pd.concat([df_original,df_single_attack], ignore_index=True)
else:
    df_original = df_single_attack
    df_original['Company'] = company
    df_original['Attack Date'] = date

df_original.to_csv(final_file_path)
df_original.head()

,Information_Privacy,Proprietary_data_loss,Business_interruption,Cyber_extorsion,Network_security,Financial_fraud,Media_content,Hardware_bricking,Post_breach_security_requirements,Reputational_damage,Data_affected_or_stolen,Alleged_attacker_or_threat_actor,Response_by_the_entity,Cyber_extortion,Note,Company,Attack Date
0,"[$6,090,909, $5,000,000,000, $5,000,000,000, $...",[],"[Tens of millions USD - $30,000,000 - $3,000,000]","[$20, tens of thousands of dollars, $35,100, $...","[$5,000, Costs associated with finding and fix...","[$2,087.5 (estimated user losses from scams le...",[],[],"[$57,000,000, $57,000,000, $57,000,000, 570000...","[$60,000,000,000 (market cap wiped off due to ...",[Personal data of over 533 million Facebook us...,"[An unnamed hacker (one claimed to be Russian,...",[Facebook stated that the data was 'old' (from...,[],[The provided text passages and economic value...,Facebook,Mar_2021
